# Capstone Feature Engineering Notebook

This notebook loads the price and Reddit sentiment datasets, computes technical indicators, aggregates sentiment, merges features, creates a classification label, and exports the final engineered dataset.

## 1. Load Data
Load price and Reddit sentiment data.

In [119]:
import pandas as pd
price_df = pd.read_csv('magnificent7_price_data.csv', parse_dates=['date'])
sentiment_df = pd.read_csv('reddit_sentiment_posts.csv', parse_dates=['date'])
print('Price data shape:', price_df.shape)
print('Sentiment data shape:', sentiment_df.shape)

Price data shape: (9821, 7)
Sentiment data shape: (1053, 5)


## 2. Add Technical Indicators
Compute SMA, EMA, RSI, MACD, Bollinger Bands, and daily returns for each ticker.

In [120]:
import numpy as np
import ta
# Compute technical indicators per ticker
def add_technical_indicators(df):
    df = df.sort_values(['ticker', 'date'])
    df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
    grouped = df.groupby('ticker')
    df['SMA_5'] = grouped['Close'].transform(lambda x: x.rolling(5).mean())
    df['SMA_10'] = grouped['Close'].transform(lambda x: x.rolling(10).mean())
    df['SMA_20'] = grouped['Close'].transform(lambda x: x.rolling(20).mean())
    df['EMA_10'] = grouped['Close'].transform(lambda x: x.ewm(span=10, adjust=False).mean())
    df['RSI_14'] = grouped['Close'].transform(lambda x: ta.momentum.RSIIndicator(x, window=14).rsi())
    macd = grouped['Close'].transform(lambda x: ta.trend.MACD(x, window_slow=26, window_fast=12, window_sign=9).macd())
    macd_signal = grouped['Close'].transform(lambda x: ta.trend.MACD(x, window_slow=26, window_fast=12, window_sign=9).macd_signal())
    df['MACD'] = macd
    df['MACD_signal'] = macd_signal
    # Bollinger Bands
    bb_high = grouped['Close'].transform(lambda x: ta.volatility.BollingerBands(x, window=20, window_dev=2).bollinger_hband())
    bb_low = grouped['Close'].transform(lambda x: ta.volatility.BollingerBands(x, window=20, window_dev=2).bollinger_lband())
    df['BB_high'] = bb_high
    df['BB_low'] = bb_low
    # Daily returns
    df['return'] = grouped['Close'].transform(lambda x: x.pct_change())
    return df

price_feat_df = add_technical_indicators(price_df)
print('Data with technical indicators shape:', price_feat_df.shape)
print('Technical features added. Example:')
display(price_feat_df.tail())

Data with technical indicators shape: (9821, 17)
Technical features added. Example:


,date,ticker,Close,High,Low,Open,Volume,SMA_5,SMA_10,SMA_20,EMA_10,RSI_14,MACD,MACD_signal,BB_high,BB_low,return
9792,2025-07-28,TSLA,325.589996,330.489990,315.690002,318.450012,112673800,322.323993,322.161996,314.944998,320.535164,52.886605,1.036268,0.278185,337.902873,291.987123,0.030152
9799,2025-07-29,TSLA,321.200012,326.250000,318.250000,325.549988,87358900,320.141998,323.203998,315.121999,320.656046,50.908711,1.041655,0.430879,338.215060,292.028937,-0.013483
9806,2025-07-30,TSLA,319.040009,324.450012,311.619995,322.179993,83931900,317.438000,322.940997,316.038499,320.362221,49.919466,0.861696,0.517042,338.207364,293.869635,-0.006725
9813,2025-07-31,TSLA,308.269989,321.369995,306.100006,319.609985,85270900,318.032001,321.826996,315.669499,318.163633,45.202915,-0.148264,0.383981,338.096128,293.242870,-0.033758
9820,2025-08-01,TSLA,302.630005,309.309998,297.820007,306.209991,88838600,315.346002,319.124997,315.033499,315.339337,42.916150,-1.387766,0.029632,338.170503,291.896495,-0.018296


## 3. Aggregate Reddit Sentiment
Aggregate Reddit posts into daily sentiment signals per ticker.

In [121]:
# Optional: Sentiment scoring with VADER
try:
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
    analyzer = SentimentIntensityAnalyzer()
    sentiment_df['sentiment'] = sentiment_df['title'].astype(str).apply(lambda x: analyzer.polarity_scores(x)['compound'])
except ImportError:
    sentiment_df['sentiment'] = np.nan

agg_sentiment = sentiment_df.groupby(['date', 'ticker']).agg(
    post_count=('title', 'count'),
    total_score=('score', 'sum'),
    total_comments=('num_comments', 'sum'),
    sentiment_avg=('sentiment', 'mean')
).reset_index()
print('Aggregated sentiment shape:', agg_sentiment.shape)
display(agg_sentiment.head())

Aggregated sentiment shape: (614, 6)


,date,ticker,post_count,total_score,total_comments,sentiment_avg
0,2022-08-17,AAPL,1,6,26,0.0000
1,2022-09-18,AAPL,1,5,11,-0.2732
2,2022-09-23,AAPL,1,1,1,0.0000
3,2022-10-13,AAPL,1,14,13,0.5423
4,2022-10-17,META,1,11,0,0.0000


## 4. Merge Price & Sentiment Features
Merge technical and sentiment features on date and ticker.

In [122]:
# Prepare for merge
if price_feat_df['date'].dtype == 'datetime64[ns]':
    price_feat_df['date'] = price_feat_df['date'].dt.date
if 'ticker' not in agg_sentiment.columns and 'Ticker' in agg_sentiment.columns:
    agg_sentiment = agg_sentiment.rename(columns={'Ticker': 'ticker'})
agg_sentiment['date'] = pd.to_datetime(agg_sentiment['date']).dt.date
# Diagnostics: check merge keys
print('Price features unique dates:', price_feat_df['date'].nunique(), 'example:', price_feat_df['date'].unique()[:5])
print('Sentiment unique dates:', agg_sentiment['date'].nunique(), 'example:', agg_sentiment['date'].unique()[:5])
print('Price features unique tickers:', price_feat_df['ticker'].unique())
print('Sentiment unique tickers:', agg_sentiment['ticker'].unique())
print('Sample price_feat_df merge keys:')
display(price_feat_df[['date','ticker']].drop_duplicates().head())
print('Sample agg_sentiment merge keys:')
display(agg_sentiment[['date','ticker']].drop_duplicates().head())
merged = pd.merge(price_feat_df, agg_sentiment, how='left', on=['date', 'ticker'])
print('Merged dataset shape:', merged.shape)
display(merged.tail(100))

Price features unique dates: 1403 example: [datetime.date(2020, 1, 2) datetime.date(2020, 1, 3)
 datetime.date(2020, 1, 6) datetime.date(2020, 1, 7)
 datetime.date(2020, 1, 8)]
Sentiment unique dates: 368 example: [datetime.date(2022, 8, 17) datetime.date(2022, 9, 18)
 datetime.date(2022, 9, 23) datetime.date(2022, 10, 13)
 datetime.date(2022, 10, 17)]
Price features unique tickers: ['AAPL' 'AMZN' 'GOOGL' 'META' 'MSFT' 'NVDA' 'TSLA']
Sentiment unique tickers: ['AAPL' 'META' 'NVDA' 'TSLA' 'GOOGL' 'MSFT' 'AMZN']
Sample price_feat_df merge keys:


,date,ticker
0,2020-01-02,AAPL
7,2020-01-03,AAPL
14,2020-01-06,AAPL
21,2020-01-07,AAPL
28,2020-01-08,AAPL


Sample agg_sentiment merge keys:


,date,ticker
0,2022-08-17,AAPL
1,2022-09-18,AAPL
2,2022-09-23,AAPL
3,2022-10-13,AAPL
4,2022-10-17,META


Merged dataset shape: (9821, 21)


,date,ticker,Close,High,Low,Open,Volume,SMA_5,SMA_10,SMA_20,...,RSI_14,MACD,MACD_signal,BB_high,BB_low,return,post_count,total_score,total_comments,sentiment_avg
9721,2025-03-11,TSLA,230.580002,237.059998,217.020004,225.309998,174896400,251.590005,268.037004,304.868000,...,24.492249,-35.583228,-29.055443,388.969486,220.766514,0.037947,6.0,52.0,380.0,-0.138283
9722,2025-03-12,TSLA,248.089996,251.839996,241.100006,247.220001,142215700,245.388004,263.766005,300.847500,...,31.963305,-35.056703,-30.255695,387.689058,214.005941,0.075939,2.0,548.0,227.0,0.000000
9723,2025-03-13,TSLA,240.679993,248.289993,232.600006,248.130005,114813500,240.834000,259.639003,296.055999,...,30.584178,-34.835789,-31.171714,385.046365,207.065633,-0.029868,1.0,4.0,65.0,0.000000
9724,2025-03-14,TSLA,249.979996,251.580002,240.729996,247.309998,100242300,238.295996,255.339001,290.757999,...,34.409292,-33.523838,-31.642139,377.443585,204.072413,0.038641,NaN,NaN,NaN,NaN
9725,2025-03-17,TSLA,238.009995,245.399994,232.800003,245.059998,111900600,241.467996,250.675002,284.866499,...,31.967596,-33.068790,-31.927469,369.038280,200.694717,-0.047884,1.0,0.0,70.0,-0.102700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9816,2025-07-28,TSLA,325.589996,330.489990,315.690002,318.450012,112673800,322.323993,322.161996,314.944998,...,52.886605,1.036268,0.278185,337.902873,291.987123,0.030152,3.0,241.0,56.0,0.062300
9817,2025-07-29,TSLA,321.200012,326.250000,318.250000,325.549988,87358900,320.141998,323.203998,315.121999,...,50.908711,1.041655,0.430879,338.215060,292.028937,-0.013483,NaN,NaN,NaN,NaN
9818,2025-07-30,TSLA,319.040009,324.450012,311.619995,322.179993,83931900,317.438000,322.940997,316.038499,...,49.919466,0.861696,0.517042,338.207364,293.869635,-0.006725,NaN,NaN,NaN,NaN
9819,2025-07-31,TSLA,308.269989,321.369995,306.100006,319.609985,85270900,318.032001,321.826996,315.669499,...,45.202915,-0.148264,0.383981,338.096128,293.242870,-0.033758,1.0,29.0,9.0,0.000000


## 5. Merge Macroeconomic Data
Merge the engineered price and sentiment features with macroeconomic indicators from FRED using the date as the key.

In [123]:
# Merge macroeconomic data from FRED
macro_df = pd.read_csv('us_macro_data_fred.csv', parse_dates=['Date'])
macro_df['date'] = macro_df['Date'].dt.date

# Diagnostics: check merge keys
print('Merged (price+sentiment) unique dates:', merged['date'].nunique(), 'example:', merged['date'].unique()[:5])
print('Macro unique dates:', macro_df['date'].nunique(), 'example:', macro_df['date'].unique()[:5])
print('Sample merged (price+sentiment) merge keys:')
display(merged[['date']].drop_duplicates().head())
print('Sample macro_df merge keys:')
display(macro_df[['date']].drop_duplicates().head())

# Merge with previous merged DataFrame (price + sentiment)
merged_macro = pd.merge(merged, macro_df.drop(columns=['Date']), how='left', on='date')
print('Merged with macroeconomic data. Shape:', merged_macro.shape)
display(merged_macro.tail(100))

Merged (price+sentiment) unique dates: 1403 example: [datetime.date(2020, 1, 2) datetime.date(2020, 1, 3)
 datetime.date(2020, 1, 6) datetime.date(2020, 1, 7)
 datetime.date(2020, 1, 8)]
Macro unique dates: 1477 example: [datetime.date(2020, 1, 1) datetime.date(2020, 1, 2)
 datetime.date(2020, 1, 3) datetime.date(2020, 1, 6)
 datetime.date(2020, 1, 7)]
Sample merged (price+sentiment) merge keys:


,date
0,2020-01-02
1,2020-01-03
2,2020-01-06
3,2020-01-07
4,2020-01-08


Sample macro_df merge keys:


,date
0,2020-01-01
1,2020-01-02
2,2020-01-03
3,2020-01-06
4,2020-01-07


Merged with macroeconomic data. Shape: (9821, 30)


,date,ticker,Close,High,Low,Open,Volume,SMA_5,SMA_10,SMA_20,...,sentiment_avg,GDP,UNRATE,CPI,FEDFUNDS,DGS10,GS10,M2,SP500,VIXCLS
9721,2025-03-11,TSLA,230.580002,237.059998,217.020004,225.309998,174896400,251.590005,268.037004,304.868000,...,-0.138283,29962.047,4.2,319.615,4.33,4.28,4.28,21656.9,5572.07,26.92
9722,2025-03-12,TSLA,248.089996,251.839996,241.100006,247.220001,142215700,245.388004,263.766005,300.847500,...,0.000000,29962.047,4.2,319.615,4.33,4.32,4.28,21656.9,5599.30,24.23
9723,2025-03-13,TSLA,240.679993,248.289993,232.600006,248.130005,114813500,240.834000,259.639003,296.055999,...,0.000000,29962.047,4.2,319.615,4.33,4.27,4.28,21656.9,5521.52,24.66
9724,2025-03-14,TSLA,249.979996,251.580002,240.729996,247.309998,100242300,238.295996,255.339001,290.757999,...,NaN,29962.047,4.2,319.615,4.33,4.31,4.28,21656.9,5638.94,21.77
9725,2025-03-17,TSLA,238.009995,245.399994,232.800003,245.059998,111900600,241.467996,250.675002,284.866499,...,-0.102700,29962.047,4.2,319.615,4.33,4.31,4.28,21656.9,5675.12,20.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9816,2025-07-28,TSLA,325.589996,330.489990,315.690002,318.450012,112673800,322.323993,322.161996,314.944998,...,0.062300,30331.117,4.2,321.500,4.33,4.42,4.39,22020.8,6389.77,15.03
9817,2025-07-29,TSLA,321.200012,326.250000,318.250000,325.549988,87358900,320.141998,323.203998,315.121999,...,NaN,30331.117,4.2,321.500,4.33,4.34,4.39,22020.8,6370.86,15.98
9818,2025-07-30,TSLA,319.040009,324.450012,311.619995,322.179993,83931900,317.438000,322.940997,316.038499,...,NaN,30331.117,4.2,321.500,4.33,4.38,4.39,22020.8,6362.90,15.48
9819,2025-07-31,TSLA,308.269989,321.369995,306.100006,319.609985,85270900,318.032001,321.826996,315.669499,...,0.000000,30331.117,4.2,321.500,4.33,4.37,4.39,22020.8,6339.39,16.72


## 6. Label Creation for Classification Task
Create binary direction label: 1 if next day's Close > today's Close, else 0.

In [124]:
# Create direction label (Step 6)
merged_macro = merged_macro.sort_values(['ticker', 'date'])
merged_macro['next_close'] = merged_macro.groupby('ticker')['Close'].shift(-1)
merged_macro['direction'] = (merged_macro['next_close'] > merged_macro['Close']).astype(int)
merged_macro = merged_macro.drop(columns=['next_close'])
print('Data with label column shape:', merged_macro.shape)
print('Label column created. Example:')
display(merged_macro[['date', 'ticker', 'Close', 'direction']].tail(100))

Data with label column shape: (9821, 31)
Label column created. Example:


,date,ticker,Close,direction
9721,2025-03-11,TSLA,230.580002,1
9722,2025-03-12,TSLA,248.089996,0
9723,2025-03-13,TSLA,240.679993,1
9724,2025-03-14,TSLA,249.979996,0
9725,2025-03-17,TSLA,238.009995,0
...,...,...,...,...
9816,2025-07-28,TSLA,325.589996,0
9817,2025-07-29,TSLA,321.200012,0
9818,2025-07-30,TSLA,319.040009,0
9819,2025-07-31,TSLA,308.269989,0


## 7. Export Engineered Dataset
Save the final engineered dataset to CSV.

In [ ]:
# Export final engineered dataset (Step 7)
final_cols = ['date', 'ticker'] + [col for col in merged_macro.columns if col not in ['date', 'ticker', 'Date', 'Ticker']]
final_df = merged_macro[final_cols]

# Forward-fill missing values within each ticker before export
final_df = final_df.sort_values(['ticker', 'date'])
final_df = final_df.groupby('ticker', as_index=False).ffill()
final_df = final_df.reset_index(drop=True)

# Back-fill any remaining missing values within each ticker
final_df = final_df.groupby('ticker', as_index=False).bfill()
final_df = final_df.reset_index(drop=True)

# Ensure ticker column is present and not index
if 'ticker' not in final_df.columns and final_df.index.name == 'ticker':
    final_df = final_df.reset_index()
if 'ticker' not in final_df.columns:
    raise ValueError('ticker column missing from final_df before export')

# Reorder columns to ensure 'date' and 'ticker' are first
final_df = final_df[['date', 'ticker'] + [col for col in final_df.columns if col not in ['date', 'ticker']]]

final_df.to_csv('final_features_dataset.csv', index=False)
print('Exported final_features_dataset.csv. Shape:', final_df.shape)
display(final_df.tail(500))

KeyError: 'ticker'